## **Preprocessing dát o VŠ učiteľoch**

In [81]:
import pandas as pd

In [82]:
def fix_table(df, university_type=None):
    """
    Fill the university names. Remove summary rows. Eventually add type of university.
    """
    # backfill / bfill: replaces NaN values with the next non-NaN value that appears in the column
    df['univerzita'] = df['univerzita'].fillna(method='backfill')
    # remove summary rows = rows without specified Faculty
    df = df.dropna(axis=0, subset='fakulta').reset_index(drop=True)
    if university_type:
        df['typ'] = university_type
    return df

In [86]:
def load_summary_table(file_path, sheet_name_fulltime, sheet_name_parttime, university_type):
    """
    Loads summary data about universities and their numbers of teachers into a dataframe.
    """
    df_fulltime = fix_table(pd.read_excel(file_path, sheet_name=sheet_name_fulltime, header=5, usecols="A:D"
                                         ).rename(columns={'Unnamed: 0': 'univerzita', 'Unnamed: 1': 'fakulta', 
                                                           'Unnamed: 2': 'spolu full-time', 'Unnamed: 3': 'zeny full-time'}),
                            university_type=university_type)
    df_parttime = fix_table(pd.read_excel(file_path, sheet_name=sheet_name_parttime, header=5, usecols="A:D"
                                         ).rename(columns={'Unnamed: 0': 'univerzita', 'Unnamed: 1': 'fakulta', 
                                                           'Unnamed: 2': 'spolu part-time', 'Unnamed: 3': 'zeny part-time'}),
                            university_type=university_type)
    merged = pd.merge(df_fulltime, df_parttime, how="outer", on=['univerzita', 'fakulta', 'typ'])
    return merged[['univerzita', 'fakulta', 'typ', 'spolu full-time', 'spolu part-time', 'zeny full-time', 'zeny part-time']]

In [87]:
public_22 = load_summary_table('../data/cvtisr/vs2022/vs2022.xls', sheet_name_fulltime='Tab16v', sheet_name_parttime='Tab17v', university_type='verejná')
display(public_22)

,univerzita,fakulta,typ,spolu full-time,spolu part-time,zeny full-time,zeny part-time
0,UNIVERZITA KOMENSKÉHO,Rímskokat.bohosl.fak.UK,verejná,21,19.0,2,3.0
1,UNIVERZITA KOMENSKÉHO,Evanj. bohosl. fak. UK,verejná,14,2.0,2,2.0
2,UNIVERZITA KOMENSKÉHO,Fa matem.fyz. a inf. UK,verejná,211,21.0,63,5.0
3,UNIVERZITA KOMENSKÉHO,Prírodovedecká fak. UK,verejná,319,48.0,139,20.0
4,UNIVERZITA KOMENSKÉHO,Fakulta managementu UK,verejná,69,2.0,39,1.0
...,...,...,...,...,...,...,...
112,VŠ MÚZICKÝCH UMENÍ,VŠMU - rektorát,verejná,3,2.0,3,2.0
113,VŠ VÝTVARNÝCH UMENÍ,Fa výtvar. umení VŠVU,verejná,118,12.0,40,4.0
114,AKADÉMIA UMENÍ,Fa dramat. umení AU,verejná,35,2.0,10,1.0
115,AKADÉMIA UMENÍ,Fa múzických umení AU,verejná,66,8.0,23,4.0


In [85]:
def add_type_of_teacher(df, type_of_teacher):
    """
    Add column for type_of_teacher.
    """
    df['typ učiteľa'] = type_of_teacher
    return df

In [77]:
def create_degrees_table(file_path, sheet_name, commitment='full-time'):
    """
    Loads data about teachers and their degrees into a dataframe.
    """
    df_prof = add_type_of_teacher(fix_table(pd.read_excel(file_path, sheet_name=sheet_name, usecols="A:B, E:G", header=5
                                                         ).rename(columns={'Unnamed: 0': 'univerzita', 'Unnamed: 1': 'fakulta', 
                                                                  'Unnamed: 4': 'spolu', 'Unnamed: 5': 'DrSc.', 'PhD.': 'CSc., PhD.'})),
                                  type_of_teacher='profesor')
    df_doc = add_type_of_teacher(fix_table(pd.read_excel(file_path, sheet_name=sheet_name, usecols="A:B, H:J", header=5
                                                        ).rename(columns={'Unnamed: 0': 'univerzita', 'Unnamed: 1': 'fakulta', 
                                                                 'Unnamed: 7': 'spolu', 'Unnamed: 8': 'DrSc.', 'PhD..1': 'CSc., PhD.'})),
                                 type_of_teacher='docent')
    df_odb = add_type_of_teacher(fix_table(pd.read_excel(file_path, sheet_name=sheet_name, usecols="A:B, K:L", header=5
                                                        ).rename(columns={'Unnamed: 0': 'univerzita', 'Unnamed: 1': 'fakulta', 
                                                                 'Unnamed: 10': 'spolu', 'PhD..2': 'CSc., PhD.'})),
                                 type_of_teacher='odborný asistent')
    df_asis = add_type_of_teacher(fix_table(pd.read_excel(file_path, sheet_name=sheet_name, usecols="A:B, M", header=5
                                                        ).rename(columns={'Unnamed: 0': 'univerzita', 'Unnamed: 1': 'fakulta', 
                                                                 'Unnamed: 10': 'spolu', 'PhD..2': 'CSc., PhD.'})),
                                 type_of_teacher='asistent')
    df_lekt = add_type_of_teacher(fix_table(pd.read_excel(file_path, sheet_name=sheet_name, usecols="A:B, N", header=5
                                                        ).rename(columns={'Unnamed: 0': 'univerzita', 'Unnamed: 1': 'fakulta', 
                                                                 'Unnamed: 12': 'spolu'})),
                                 type_of_teacher='lektor')
    df_host = add_type_of_teacher(fix_table(pd.read_excel(file_path, sheet_name=sheet_name, usecols="A:B, O", header=5
                                                        ).rename(columns={'Unnamed: 0': 'univerzita', 'Unnamed: 1': 'fakulta', 
                                                                 'Unnamed: 13': 'spolu'})),
                                 type_of_teacher='hosťujúci profesor')
    concatenated = pd.concat([df_prof, df_doc, df_odb, df_asis, df_lekt, df_host], axis=0, join='outer'
                            ).fillna(0).astype({'DrSc.': int, 'CSc., PhD.': int, 'spolu': int})
    concatenated['iný/žiadny titul'] = concatenated['spolu'] - concatenated['DrSc.'] - concatenated['CSc., PhD.']
    concatenated['úväzok'] = commitment
    
    return concatenated[['univerzita', 'fakulta', 'typ učiteľa', 'úväzok', 'DrSc.', 'CSc., PhD.', 'iný/žiadny titul', 'spolu']].reset_index(drop=True)
    

In [80]:
degrees_fulltime = create_degrees_table(file_path='../data/cvtisr/vs2022/vs2022.xls', sheet_name='Tab16v', commitment='full-time')
degrees_parttime = create_degrees_table(file_path='../data/cvtisr/vs2022/vs2022.xls', sheet_name='Tab17v', commitment='part-time')

degrees_22 = pd.concat([degrees_fulltime, degrees_parttime]).reset_index(drop=True)
display(degrees_22)

,univerzita,fakulta,typ učiteľa,úväzok,DrSc.,"CSc., PhD.",iný/žiadny titul,spolu
0,UNIVERZITA KOMENSKÉHO,Rímskokat.bohosl.fak.UK,profesor,full-time,0,6,0,6
1,UNIVERZITA KOMENSKÉHO,Evanj. bohosl. fak. UK,profesor,full-time,0,4,0,4
2,UNIVERZITA KOMENSKÉHO,Fa matem.fyz. a inf. UK,profesor,full-time,17,26,0,43
3,UNIVERZITA KOMENSKÉHO,Prírodovedecká fak. UK,profesor,full-time,17,32,0,49
4,UNIVERZITA KOMENSKÉHO,Fakulta managementu UK,profesor,full-time,0,15,0,15
...,...,...,...,...,...,...,...,...
1285,VŠ MÚZICKÝCH UMENÍ,VŠMU - rektorát,hosťujúci profesor,part-time,0,0,0,0
1286,VŠ VÝTVARNÝCH UMENÍ,Fa výtvar. umení VŠVU,hosťujúci profesor,part-time,0,0,0,0
1287,AKADÉMIA UMENÍ,Fa dramat. umení AU,hosťujúci profesor,part-time,0,0,0,0
1288,AKADÉMIA UMENÍ,Fa múzických umení AU,hosťujúci profesor,part-time,0,0,0,0
